In [2]:
import psycopg2 as ps
from env_vars import *

In [3]:
# def reconnect():
#     # create connection and cursor    
#     conn = ps.connect(host=POSTGRES_ADDRESS,
#                       database=POSTGRES_DBNAME,
#                       user=POSTGRES_USERNAME,
#                       password=POSTGRES_PASSWORD,
#                       port=POSTGRES_PORT)
#     cur = conn.cursor()
#     return cur

In [7]:
def reconnect:
    conn = ps.connect(host=POSTGRES_ADDRESS,
                  database=POSTGRES_DBNAME,
                  user=POSTGRES_USERNAME,
                  password=POSTGRES_PASSWORD,
                  port=POSTGRES_PORT)
    cur = conn.cursor()

In [42]:
# def run_query():
#     try:
#         cur.execute(query)
#     except Exception as e:
#         print (e.message)
#         return reconnect()

In [43]:
# run_query()

In [ ]:
# query = """SELECT * FROM users
#             WHERE schemaname != 'pg_catalog'
#             AND schemaname != 'information_schema';"""
# cur.execute(query)
# cur.fetchall()

In [4]:
query = """SELECT * FROM users;"""

cur.execute(query)
cur.fetchall()

[]

In [9]:
query = """SELECT * FROM recommendations"""

cur.execute(query)
cur.fetchall()

[]

In [ ]:
# cur.execute("""CREATE TABLE users
#                 (ID integer, 
#                 display_name varchar(50)
#                 );""")
# # Commit table creation
# conn.commit()

In [8]:
cur.execute("""CREATE TABLE recommendations
                (id serial PRIMARY KEY,
                userID varchar(50), 
                songID varchar(50),
                recDate DATE 
                )""")
# Commit table creation
conn.commit()

In [18]:
cur.execute("""CREATE TABLE account(
               user_id serial PRIMARY KEY,
               username VARCHAR (50) UNIQUE NOT NULL,
               password VARCHAR (50) NOT NULL,
               email VARCHAR (355) UNIQUE NOT NULL,
               created_on TIMESTAMP NOT NULL,
               last_login TIMESTAMP
               );""")
# Commit table creation
conn.commit()

In [ ]:
cursor.execute("""CREATE TABLE survival(
id SERIAL PRIMARY KEY,
survived integer)""")